Only run if in Colab!

In [ ]:
!git clone https://github.com/ratinac-nikola/rbm.git
%cd rbm
%pip install -r requirements.txt

In [ ]:
%pwd

### Initialization

In [ ]:
from sklearn.model_selection import train_test_split
import torch
import config

if torch.cuda.is_available():
    print("Using GPU")
    device = torch.device('cuda')
else:
    print("Using CPU")
    device = "cpu"

In [ ]:
from dataset import Dataset
dataset = Dataset(device=device)
tensorData = dataset.getDatasetAsTensor()
trainset, testset = train_test_split(tensorData, train_size=0.05)
trainset.shape, testset.shape

In [ ]:
from rbm import RBM
rbm = RBM(tensorData.shape[1], n_hidden=10, device=device)

In [ ]:
mse = torch.nn.MSELoss()

for epoch in range(1, config.epochs + 1):
    i = 1
    recon_err = []
    c = 1
    for case in trainset:
        print(c)
        v0 = case
        vk = case
        c += 1
        for k in range(2):
            vk = rbm.reconstruct(vk)

            vk[v0.sum(dim=1) == 0] = v0[v0.sum(dim=1) == 0]
        
        rbm.train(v0, vk)
        
        vv = rbm.reconstruct(v0)
        vv[v0.sum(dim=1) == 0] = v0[v0.sum(dim=1) == 0]
        
        v0 = torch.argmax(v0, dim=1).float()
        vv = torch.argmax(vv, dim=1).float()
        rmse = torch.sqrt(mse(v0,vv)).item()
        
        recon_err += [rmse]
        
    print(f"Epoch {epoch} done. Mean reconstruction RMSE: {sum(recon_err)/len(recon_err)}")
    
    recon_err = []
    for case in testset:
        rec = rbm.reconstruct(case)
        rec[case.sum(dim=1) == 0] = case[case.sum(dim=1) == 0]
        case = torch.argmax(case, dim=1).float()
        rec = torch.argmax(rec, dim=1).float()
        
        recon_err += [torch.sqrt(mse(case,rec)).item()]
    
    print(f"Testset reconstruction: {sum(recon_err)/len(recon_err)}")
        
        
    
